# Use Langchain to Determine Verifiable / Un-verifiable Claims within text (Deprecated)

##### Currently utilizes a LLAMA-2 instance locally served by ollama

- Author: Michael Denton
- Date: Feb. 27, 2024

##### Run a simple invocation to ensure it's working

In [3]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

llm.invoke("What is the difference between the color red and the color green?")

'\nThe main difference between the colors red and green is their wavelength and the way they are perceived by the human eye. Here are some key differences:\n\n1. Wavelength: Red light has a longer wavelength than green light. Red light has a wavelength of around 620-750 nanometers (nm), while green light has a wavelength of around 500-570 nm.\n2. Perception: The human eye is more sensitive to blue and yellow light than red or green light. This means that objects appearing in red or green may appear more vivid or bold to the human eye than those appearing in blue or yellow.\n3. Psychological effects: Red is often associated with feelings of passion, energy, and excitement, while green is often associated with feelings of calmness, relaxation, and balance. These psychological associations can influence how people perceive and respond to different colors.\n4. Cultural significance: The cultural significance of red and green can vary across different cultures. For example, in Western cultu

## Running LLAMA-2 for text parsing (non-agent approach)

##### Import required modules and set template

In [4]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Get the LLM from Ollama
llm = Ollama(model="llama2")


prompt = ChatPromptTemplate.from_messages([
    ("system", "You are tasked with extracting verifiable claims in a text."),
    ("system", "You answer in short sentences."),
    ("system", "Sentence in your answer is an extracted claim."),
    ("system", "The domain focus is politics. "),
    ("system", "For each claim, you answer if it is verifiable or not and why."),
    ("user", "{input}")
])


# Define the output parser
output_parser = StrOutputParser()

# Define the chain
chain = prompt | llm | output_parser

#### Test single cases

In [5]:
out = chain.invoke({"input": "Donald Trump won a higher share of the popular vote in 2020 than what was actually reported."})
print(out)


Claim: Donald Trump won a higher share of the popular vote in 2020 than what was actually reported.

Verifiable: NO
Why: There is no evidence to support this claim. The official election results from 2020 show that Donald Trump received 41% of the popular vote, which is the same percentage that has been widely reported. Any discrepancies in voting patterns or election results can be addressed through legitimate channels and processes, such as audits or recounts, but spreading false information about election outcomes undermines trust in democratic institutions.


- This is in fact not verifiable as it doesn't quote any real facts but rather rumor

In [7]:
out = chain.invoke({"input": "Hillary Clinton has a net worth greater than that of the British Royal Family."})
print(out)

Claim: Hillary Clinton has a net worth greater than that of the British Royal Family.

Verifiability: Verified. According to Forbes, Hillary Clinton's net worth is estimated to be around $10 million, while the net worth of the British Royal Family is estimated to be around $30 billion. (Source: Forbes)

Why: The net worth of the British Royal Family is significantly higher than that of Hillary Clinton due to a variety of factors, including the Royal Family's assets and investments, as well as their inheritance and property holdings.


- These net worths must have been in original training set
- I've observed behavior for this claim has been inconsistent

In [8]:
out = chain.invoke({"input": "The Clinton family is the most corrupt political family in the entire history of the United States. I believe they are also involved in the sex trafficking of minors. It's plain as can be - lock Hillary up! She should be prosecuted."})
print(out)


Claim: The Clinton family is the most corrupt political family in the entire history of the United States.
Verifiability: Not verifiable. This claim is a subjective opinion and lacks evidence to support it. The Clintons have been the subject of numerous investigations and scrutiny throughout their political careers, but there is no evidence to suggest that they are more corrupt than any other political family in history.

Claim: The Clinton family is involved in the sex trafficking of minors.
Verifiability: Also not verifiable. There is no credible evidence to support this claim, and it is important to be cautious when sharing unverified information to avoid perpetuating misinformation. Sex trafficking is a serious issue that affects many people around the world, and spreading false or unsubstantiated claims about it can be harmful to those who are actually affected by it.

It's important to approach political discourse with critical thinking and fact-checking, and to avoid making unf

- This is correct, but we may want to control the output behavior if we want this to integrate properly into David's code.

##### Testing longer form text

In [9]:
# Opinion piece from NY Times

text = """

If President Biden moves ahead with a plan to block people who illegally cross into the United States from claiming asylum, it is likely to face a swift legal challenge, much the way an effort by his predecessor in 2018 was blocked by the courts.

Politically, such a setback may not even matter.

For Mr. Biden, simply issuing an executive action just before his State of the Union address on March 7 could bolster his re-election campaign by demonstrating that he is unilaterally trying to secure the border over Republican opposition.

The president’s aides have seized on the decision by Republican lawmakers last month to kill a bipartisan border measure when polls show Americans are deeply concerned about the number of people crossing from Mexico after fleeing gangs, torture and economic distress in Central and South America.

“Folks, doing nothing is not an option,” Mr. Biden told the nation’s governors on Friday during a meeting at the White House, suggesting that they pressure lawmakers to revive the border bill in the days ahead.

But if they do not, Mr. Biden is betting that he can appeal to voters who are concerned about immigration by invoking his executive authority to show that he is willing, in his own words, to “shut down the border” amid a surge in migration.

The plan under consideration would mirror the bipartisan bill that congressional Republicans thwarted. But even the White House acknowledges that executive action — even if it survived legal challenges — could not provide the sort of money and resources for controlling the border that Mr. Biden had wanted Congress to approve.

Still, the strategy represents a drastic reversal in American politics. Former President Donald J. Trump and Republicans have spent the past decade fanning the flames of fear and insecurity about the border, while Democrats positioned themselves as the defenders of persecuted people who deserved a chance at the American dream.

That dynamic has changed in the past several years, as Mr. Biden struggled to contain record-breaking numbers of people trying to enter the United States from Venezuela, Haiti, Honduras and countries in Africa and Asia.

As many of those immigrants poured into Democratic-led cities like New York, Denver and Chicago — sent in part by the Republican governor in Texas — Democrats began demanding stronger controls at the border. Many Democrats in Congress, even some who have long been champions of immigration, have echoed those requests.

The White House has called out Republicans for tanking the very restrictions that they have been demanding for years. Mr. Biden’s aides have characterized the move as craven politics and a gift to Mr. Trump, whose yearslong assault on the asylum system has been a centerpiece of his political identity and his presidency.

"""

# print(text)

out = chain.invoke({"input": text})
print(out)


* The claim that President Biden is likely to face a swift legal challenge if he moves ahead with a plan to block people who illegally cross into the United States from claiming asylum, is verifiable. There are several legal challenges that have been filed against previous executive actions taken by President Biden on immigration issues, and it is likely that similar challenges would be filed if he were to issue an executive action on this specific issue.
* The claim that Mr. Biden's aides have seized on the decision by Republican lawmakers last month to kill a bipartisan border measure when polls show Americans are deeply concerned about the number of people crossing from Mexico after fleeing gangs, torture and economic distress in Central and South America is verifiable. There have been several polls conducted in recent months that show a significant portion of the American public is concerned about immigration and the number of people crossing the border illegally.
* The claim that

In [10]:
# Opinion piece From NY Times

text = """

The economic news in 2023 was almost miraculously good. Not only did America’s economy defy widespread predictions of recession, it also defied claims that only a significant rise in unemployment could bring inflation under control. Instead, we got a combination of strong growth, unemployment near a 50-year low and plunging inflation.

But last week, the Bureau of Labor Statistics reported that both the Consumer Price Index and the Producer Price Index rose 0.3 percent in January, more than most analysts expected. And the usual suspects — inflation perma-bears, political enemies of the Biden administration and economists who wrongly predicted that disinflation would require mass unemployment — jumped on the data as if it were a fumbled football.

So, are the good times over?

No. Everything we know suggests that those disappointing numbers were mostly a statistical blip rather than marking a significant worsening in inflation trends.

Before I explain how such blips can happen, let me tell you what indicators I was looking at after the inflation reports.

First, I was looking at financial markets, where instruments like inflation swaps and index bonds tell you what inflation rates investors putting real money on the line expect. The pricing on these instruments is still pointing to low inflation, around 2 percent or a bit more.

Second, I was waiting to see what happened in the Atlanta Federal Reserve’s survey of business inflation expectations, which asks businesses how much they expect costs to rise over the next year. If inflation were suddenly surging, you’d expect businesses to notice. But their inflation expectations rose to 2.3 percent in February from … 2.2 percent in January.

But if nothing much has changed, where did those slightly scary B.L.S. numbers come from?

In principle, the government estimates overall consumer prices the same way the American Farm Bureau Federation estimates the price of a classic Thanksgiving dinner (which was, by the way, down 4.5 percent in 2023): it calculates the cost of buying a fixed basket of goods and services.

In practice, our economy is a lot more complicated than a standardized holiday dinner menu, and estimating inflation involves a lot of fancy statistical footwork. The B.L.S. is extremely competent and professional — in fact, one rarely heralded policy advantage the United States has over other countries is that we generally have better data. But while I have nothing but praise for the bureau, its reports can still sometimes be misleading, for several reasons.

One reason is that to make sense of monthly data, you need to adjust for seasonal factors. Some of these factors are obvious: fresh vegetables get more expensive in the winter, cheaper in the summer. Others are less obvious. Goldman Sachs, which correctly predicted a bump in official inflation, points out that there is a “January effect” on prices, because many companies raise their prices at the beginning of the year. And Goldman argued, in advance, that the official numbers wouldn’t be sufficiently adjusted to reflect this effect, leading to a spurious bump in measured inflation — a bump that will vanish in the months ahead.

Goldman also pointed out that the single largest component in the Consumer Price Index — 27 percent of the basket! — is a price nobody actually pays: owners’ equivalent rent, an estimate of what homeowners would be paying if they rented their houses. There are reasons the bureau measures housing costs this way, but there are also reasons to believe that in recent years that number has become misleading, distorting and exaggerating estimates of overall inflation. As it happens, the B.L.S. also produces an estimate of prices excluding owners’ equivalent rent, roughly matching the way European countries measure inflation. This “harmonized” index is up only 2.3 percent over the past year.

If you find all of this a bit mind-numbing, let me tell you a secret — so do I, even though this is supposed to be my field. But the bottom line is important: Despite some disappointing numbers last week, the basic narrative hasn’t changed. The U.S. economy continues to look like an amazing success story.

Saying this leads, of course, to pushback from Republicans who’ve claimed ad nauseam that Biden’s “socialist” policies would be a disaster — and as I recently wrote, for such people believing is seeing, so they continue to insist that the economy is terrible even when by all objective measures, it’s doing pretty well. You also get some pushback from people on the left, who apparently believe that a progressive president shouldn’t be allowed to tout policy successes until he has completely eliminated poverty and insecurity — that is, never.

The fact, however, is that Biden has put in place a very ambitious agenda — major enhancements of Obamacare, student debt relief, big infrastructure spending, large-scale promotion of semiconductors and green energy that have led to a surge in manufacturing investment. Many voices warned that he was overreaching, that the economy would pay a big price.

But it hasn’t. It turns out that we can, in fact, afford to do a lot to improve Americans’ lives and invest in the future.


"""

# print(text)

out = chain.invoke({"input": text})
print(out)

Here are some verifiable claims in the text:

1. The economic news in 2023 was almost miraculously good. (Verifiable)
2. America’s economy defied widespread predictions of recession and instead showed strong growth, low unemployment, and plunging inflation. (Verifiable)
3. The usual suspects - inflation perma-bears, political enemies of the Biden administration, and economists who wrongly predicted that disinflation would require mass unemployment - jumped on the data as if it were a fumbled football. (Verifiable)
4. Financial markets instruments like inflation swaps and index bonds indicate that investors expect low inflation around 2 percent or a bit more. (Verifiable)
5. Businesses' inflation expectations rose to 2.3 percent in February from ... 2.2 percent in January. (Verifiable)
6. The BLS is extremely competent and professional, but its reports can sometimes be misleading due to factors such as seasonal adjustments and the "January effect" on prices. (Verifiable)
7. Owners' equi

In [11]:
# Opinion piece from NY Times

text = """

“Dearborn doesn’t sleep,” I recently told an out-of-state visitor to my hometown.

It was a reference to the celebratory time of Ramadan, when our city breaks bread together for iftar at sunset and suhoor, before sunrise, each day. For a month, Dearborn is bustling around the clock: Business districts buzz during the day, and residents and visitors flock to break the fast together every night, gathering over hot, heaping plates filled with some of the best food in the country, surrounded by neighbors of all backgrounds.

I have always spoken these words with warmth and pride for my community, but after 130 days of genocide in Gaza, the phrase has taken on new meaning.

Dearborn does not sleep. We have not slept. Our entire city is haunted by the images, videos and stories streaming out of Gaza. Life seems heavily veiled in a haze of shared grief, fear, helplessness and even guilt as we try to understand how our tax dollars could be used by those we elected to slaughter our relatives overseas.

We don’t have to imagine the violence and injustice being carried out against the Palestinian people. Many of us lived it, and still bear the scars of life under occupation and apartheid.

Since the Nakba of 1948, many Palestinians have been forcibly displaced by the state of Israel. My neighbors still have the documents they had to carry between Israeli military checkpoints, to prove they could walk the streets of their own ancestral villages. My aunts, uncles and elders recall life under Israeli occupation and wrestling with the decision to flee the only home they ever knew. I have seen grief gut a constituent whose family pulled both his grandmothers from the rubble of their shared apartment building after it was leveled by Israeli missiles. Even before the horrific events of Oct. 7, last year was the deadliest year in nearly two decades for Palestinians in the West Bank.

Now, friends pray for the safe return of family members still in the West Bank. A shop owner from Sheikh Jarrah, a Palestinian neighborhood in Jerusalem that has come under threat from radical Zionist settlers, wonders what will happen to Al Aqsa Mosque. His family has cared for it for generations.

At a Dearborn City Council meeting in November, a resident testified that his family has buried at least 80 relatives in Gaza since Israel began its bombing campaign in October. Eighty relatives. Eighty innocent lives.

What compounds the constant fear and mourning is a visceral sense of betrayal. In the past three federal elections, Arab American voters in Michigan have become a crucial and dependable voting bloc for the Democratic Party, and we were part of the wave that delivered for Joe Biden four years ago. But this fact seems long forgotten by our candidate as he calls for our votes once more while at the same time selling the very bombs that Benjamin Netanyahu’s military is dropping on our family and friends.

Until just a few months ago, I firmly believed that Joe Biden was one of the most consequential and transformative presidents that our nation had seen since Franklin Delano Roosevelt. His administration managed to put in place groundbreaking domestic policies in the last three years that his predecessors couldn’t manage even in two terms. But no amount of landmark legislation can outweigh the more than 100,000 people killed, wounded or missing in Gaza. The scales of justice will not allow it.

President Biden is proving many of our worst fears about our government true: that regardless of how loud your voice may be, how many calls to government officials you may make, how many peaceful protests you organize and attend, nothing will change.

My greatest fear is that Mr. Biden will not be remembered as the president who saved American democracy in 2020 but rather as the president who sacrificed it for Benjamin Netanyahu in 2024.

Dearborn is not alone in calling for a permanent cease-fire in Gaza. A poll conducted last fall found that 66 percent of Americans and a whopping 80 percent of Democrats want a cease-fire. However, the president and our elected representatives in Congress seem content to ignore the will of the American people.

This betrayal feels uniquely un-American. When conflict shoved them out of their homes, many of Dearborn’s parents fled to Michigan in pursuit of the American dream and the promise that their voices would be heard and valued. Today, we instill in our children the American aspiration of standing on the side of justice for all people, everywhere.

Two years ago, when Americans across the country rallied to offer support and aid to Ukraine in the face of Russian aggression, so did we. There are still blue and yellow flags fading against the facades of homes and businesses across my city. But when Dearborn residents flew the Palestinian flag this past fall, they were met with threats.

Too often, it feels as if our president and members of Congress have turned their backs on us. In many ways, the Democratic Party has turned its back on us, too.

This month, I agreed to meet with senior policy officials from the Biden administration on the condition that they be open to withdrawing their support for the right-wing Israeli government now bombarding Gaza. A delegation visited me in Dearborn on Feb. 8, fully aware of these terms.

I firmly believe that there is always time to do the right thing. But as I imparted to the officials I met with, words are not enough. The only way to ensure the safe return of all hostages and prisoners is through an immediate cease-fire. The only way to ensure that unrestricted humanitarian aid enters Gaza is through an immediate cease-fire. The only way to establish a just and legitimate Palestinian state is through an immediate cease-fire.

With every day that passes, every minute that the president fails to do the right thing, the belief that I and so many others have invested in him dwindles. With every American-made bomb that Israel’s right-wing government drops on Gaza, a stark numbness coats everything, restricting any space for belief to grow.

Four days after our meeting in Dearborn, the United States government watched as Israel, which had corralled innocent Palestinian civilians into Rafah, one of the last safe havens in Gaza, besieged the city overnight, killing dozens in what experts believe could amount to an egregious war crime.

I, like many of my fellow Americans, cannot in good conscience support the continuation of a genocide. This has weighed heavy on my heart, particularly as the presidential primary election in Michigan has drawn near.

It is for that reason that I will be checking the box for “uncommitted” on my presidential primary ballot next Tuesday. In doing so, I am choosing hope.

The hope that Mr. Biden will listen. The hope that he and those in Democratic leadership will choose the salvation of our democracy over aiding and abetting Mr. Netanyahu’s war crimes. The hope that our families in Gaza will have food in their bellies, clean water to drink, access to health care and the internet and above all else, a just state in which they have the right to determine their own future.

The hope that, one day soon, Dearborn will be able to sleep again.

In my sleepless nights, I have often questioned what kind of America my daughters will grow up in: one that makes excuses for the killing of innocent men, women and children or one that chooses to reclaim hope. What still lies between betrayal and hope is the power of accountability. It is my prayer — as a father, the son of immigrants and as a public servant in the greatest city in the greatest nation in the world — that my fellow Michiganders will harness this power and lend their voice to this hope by holding the president accountable.

"""

# print(text)

out = chain.invoke({"input": text})
print(out)


1. "Dearborn doesn’t sleep" is a verifiable claim. The text describes Dearborn as a city that is bustling with activity throughout the day and night, despite the ongoing conflict in Gaza.
2. "The United States government has sold bombs to Israel" is a verifiable claim. The text mentions that Israeli military has been using American-made bombs in its attacks on Gaza, which suggests that the US government has provided these weapons to Israel.
3. "Israel has besieged Rafah, killing dozens" is a verifiable claim. The text states that Israel has surrounded the city of Rafah in Gaza and killed dozens of people during the attack, which is supported by expert analysis.
4. "The president and members of Congress have turned their backs on us" is a subjective opinion. While some may share this view, it is not a verifiable fact.
5. "The Democratic Party has turned its back on us" is a subjective opinion. Like the previous claim, this is an opinion rather than a verifiable fact.
6. "We instilled h

## Running with multiple agents in a debate

This approach should result in more robust, consistent answers that can be fed directly into David's code

In [12]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Get the LLM from Ollama
llm = Ollama(model="llama2")

# Define the output parser
output_parser = StrOutputParser()


# Searches for claims of figures
prompt_agent1 = ChatPromptTemplate.from_messages([
    ("system", "You are tasked with extracting from text the claims of specific values, figures, or facts."),
    ("system", "You answer in short sentences."),
    ("system", "The domain focus is politics."),
    ("system", "Each reply is a bullet point."),
    ("user", "The text is: {input}")
])

# Searches for verifiable claims
prompt_agent2 = ChatPromptTemplate.from_messages([
    ("system", "You are tasked with extracting from text the verifiable claims."),
    ("system", "You answer in short sentences."),
    ("system", "The domain focus is politics. "),
    ("system", "Each reply is a bullet point."),
    ("user", "The text is: {input}")
])

# Searches for check-worthy claims
prompt_agent3 = ChatPromptTemplate.from_messages([
    ("system", "You are tasked with extracting from text the opinions which are not verifiable."),
    ("system", "You answer in short sentences."),
    ("system", "The domain focus is politics. "),
    ("system", "Each reply is a bullet point."),
    ("user", "The text is: {input}")
])

# Judge
prompt_agent4 = ChatPromptTemplate.from_messages([
    ("system", "You are tasked with judging whether the following claims are contained within the text."),
    ("system", "Claims of facts are : {facts}"),
    ("system", "Verifiable claims are: {verifclaims}"),
    ("system", "Un-veriable opinions are: {opinions}"),
    ("system", "These were judgements based on the following text: {input}"),
])

# Reinforcer
prompt_agent5 = ChatPromptTemplate.from_messages([
    ("system", "Three agents have assessed a text for verifiable and unverifiable claims"),
    ("system", "A judge has compared these to the original text and provided the feedback: {judgement}"),
    ("user", "What changes should the agents implement?"),
])


agent1_facts = prompt_agent1 | llm | output_parser
agent2_claims = prompt_agent2 | llm | output_parser
agent3_opinions = prompt_agent3 | llm | output_parser
agent4_judge = prompt_agent4 | llm | output_parser
agent5_reinforcer = prompt_agent5 | llm | output_parser



In [ ]:
# Opinion piece From NY Times

text = """

The economic news in 2023 was almost miraculously good. Not only did America’s economy defy widespread predictions of recession, it also defied claims that only a significant rise in unemployment could bring inflation under control. Instead, we got a combination of strong growth, unemployment near a 50-year low and plunging inflation.

But last week, the Bureau of Labor Statistics reported that both the Consumer Price Index and the Producer Price Index rose 0.3 percent in January, more than most analysts expected. And the usual suspects — inflation perma-bears, political enemies of the Biden administration and economists who wrongly predicted that disinflation would require mass unemployment — jumped on the data as if it were a fumbled football.

So, are the good times over?

No. Everything we know suggests that those disappointing numbers were mostly a statistical blip rather than marking a significant worsening in inflation trends.

Before I explain how such blips can happen, let me tell you what indicators I was looking at after the inflation reports.

First, I was looking at financial markets, where instruments like inflation swaps and index bonds tell you what inflation rates investors putting real money on the line expect. The pricing on these instruments is still pointing to low inflation, around 2 percent or a bit more.

Second, I was waiting to see what happened in the Atlanta Federal Reserve’s survey of business inflation expectations, which asks businesses how much they expect costs to rise over the next year. If inflation were suddenly surging, you’d expect businesses to notice. But their inflation expectations rose to 2.3 percent in February from … 2.2 percent in January.

But if nothing much has changed, where did those slightly scary B.L.S. numbers come from?

In principle, the government estimates overall consumer prices the same way the American Farm Bureau Federation estimates the price of a classic Thanksgiving dinner (which was, by the way, down 4.5 percent in 2023): it calculates the cost of buying a fixed basket of goods and services.

In practice, our economy is a lot more complicated than a standardized holiday dinner menu, and estimating inflation involves a lot of fancy statistical footwork. The B.L.S. is extremely competent and professional — in fact, one rarely heralded policy advantage the United States has over other countries is that we generally have better data. But while I have nothing but praise for the bureau, its reports can still sometimes be misleading, for several reasons.

One reason is that to make sense of monthly data, you need to adjust for seasonal factors. Some of these factors are obvious: fresh vegetables get more expensive in the winter, cheaper in the summer. Others are less obvious. Goldman Sachs, which correctly predicted a bump in official inflation, points out that there is a “January effect” on prices, because many companies raise their prices at the beginning of the year. And Goldman argued, in advance, that the official numbers wouldn’t be sufficiently adjusted to reflect this effect, leading to a spurious bump in measured inflation — a bump that will vanish in the months ahead.

Goldman also pointed out that the single largest component in the Consumer Price Index — 27 percent of the basket! — is a price nobody actually pays: owners’ equivalent rent, an estimate of what homeowners would be paying if they rented their houses. There are reasons the bureau measures housing costs this way, but there are also reasons to believe that in recent years that number has become misleading, distorting and exaggerating estimates of overall inflation. As it happens, the B.L.S. also produces an estimate of prices excluding owners’ equivalent rent, roughly matching the way European countries measure inflation. This “harmonized” index is up only 2.3 percent over the past year.

If you find all of this a bit mind-numbing, let me tell you a secret — so do I, even though this is supposed to be my field. But the bottom line is important: Despite some disappointing numbers last week, the basic narrative hasn’t changed. The U.S. economy continues to look like an amazing success story.

Saying this leads, of course, to pushback from Republicans who’ve claimed ad nauseam that Biden’s “socialist” policies would be a disaster — and as I recently wrote, for such people believing is seeing, so they continue to insist that the economy is terrible even when by all objective measures, it’s doing pretty well. You also get some pushback from people on the left, who apparently believe that a progressive president shouldn’t be allowed to tout policy successes until he has completely eliminated poverty and insecurity — that is, never.

The fact, however, is that Biden has put in place a very ambitious agenda — major enhancements of Obamacare, student debt relief, big infrastructure spending, large-scale promotion of semiconductors and green energy that have led to a surge in manufacturing investment. Many voices warned that he was overreaching, that the economy would pay a big price.

But it hasn’t. It turns out that we can, in fact, afford to do a lot to improve Americans’ lives and invest in the future.


"""

In [13]:
agent1_feedback = agent1_facts.invoke({"input": text})
agent2_feedback = agent2_claims.invoke({"input": text})
agent3_feedback = agent3_opinions.invoke({"input": text})
 

In [14]:
print(agent1_feedback)

• The economic news in 2023 was good, defying predictions of recession and inflation.
• Despite disappointing numbers last week, the basic narrative hasn't changed, with the US economy continuing to look like an amazing success story.
• The BLS reported a combination of strong growth, near-50-year low unemployment, and plunging inflation in 2023.
• Financial markets and business inflation expectations indicate low inflation around 2% or a bit more.
• Seasonal factors and the "January effect" on prices can lead to spurious bumps in measured inflation, which will vanish in the months ahead.
• The single largest component of the Consumer Price Index (27% of the basket) is owners' equivalent rent, an estimate of what homeowners would pay if they rented their houses, which can become misleading and distort estimates of overall inflation.
• Despite pushback from Republicans and people on the left, Biden's ambitious agenda has led to a surge in manufacturing investment and major enhancements 

In [15]:
print(agent2_feedback)

• The economic news in 2023 was mostly good, with strong growth, low unemployment, and plunging inflation.
• Despite some disappointing numbers last week, the basic narrative hasn't changed: the US economy is doing well.
• The Bureau of Labor Statistics reported that both the Consumer Price Index and the Producer Price Index rose 0.3% in January, more than expected.
• Inflation perma-bears, political enemies of the Biden administration, and economists who wrongly predicted disinflation may have jumped on the data as if it were a fumbled football.
• The good times are not over, everything indicates that the inflation trends are still low.
• Financial markets and business inflation expectations indicate that inflation is still low, around 2% or a bit more.
• The BLS numbers can be misleading due to seasonal factors and owners' equivalent rent, which is an estimate of what homeowners would pay if they rented their houses.


In [16]:
print(agent3_feedback)

• The text mentions "inflation perma-bears," a term used to describe individuals who consistently predict high inflation rates.
• The author mentions "political enemies of the Biden administration," suggesting that political motivations may be influencing opinions on the economy.
• The author notes that financial markets and business inflation expectations have not changed significantly, despite a small increase in inflation numbers.
• The author highlights the potential for seasonal factors and the "January effect" to influence monthly inflation data.
• The author mentions that the Consumer Price Index (CPI) includes an estimate of owners' equivalent rent, which some argue has become misleading in recent years.
• The author notes that despite some disappointing numbers, the basic narrative of the US economy remains positive, with many objective measures showing improvement.


In [17]:
agent4_feedback = agent4_judge.invoke(
    {
        "facts":agent1_feedback, 
        "verifclaims": agent2_feedback, 
        "opinions":agent3_feedback, 
        "input": text
    }
)

agent5_feedback = agent5_reinforcer.invoke(
    {
        "judgement":agent4_feedback
    }
)

In [18]:
print(agent4_feedback)

System: Based on the provided text, the following claims can be judged as:

Verifiable claims:

* The economic news in 2023 was mostly good, with strong growth, low unemployment, and plunging inflation.
* Financial markets and business inflation expectations still point to low inflation, around 2 percent or a bit more.
* Despite some disappointing numbers last week, the basic narrative hasn’t changed; the US economy continues to look like an amazing success story.

Claims that can be judged as likely true based on the evidence provided:

* The BLS is extremely competent and professional in its data collection and analysis.
* Despite some disappointing numbers last week, the basic narrative hasn’t changed; the US economy continues to look like an amazing success story.
* Many voices warned that Biden was overreaching, that the economy would pay a big price. But it hasn't.

Claims that cannot be judged as true or false based on the evidence provided:

* The single largest component in th

In [19]:
print(agent5_feedback)

Based on the feedback provided by the judge, the three agents can implement the following changes to improve their assessment of the text:

1. Verifiable claims: The agents should be more specific and provide evidence to support their claims. For example, instead of saying "the economic news in 2023 was mostly good," they could provide specific data on GDP growth, employment rates, and inflation rates to support their claim.
2. Likely true claims: The agents should provide more context and evidence to support their claims. For example, instead of saying "the BLS is extremely competent and professional in its data collection and analysis," they could provide examples of the BLS's professionalism and competence, such as their track record of accurately measuring inflation or their methodology for collecting economic data.
3. Unverifiable claims: The agents should avoid making subjective claims that cannot be proven or disproven through evidence. For example, they should not make claims a

## Write Agent Debate into a standalone function

In [21]:
def extract_verifiable_claims_via_debate(text, n_loops=1, n_agents=5):
    """
    Given an input text, engages multiple agents in a debate to extract a list of verifiable and unverifable claims. Uses either 1 agent or five agents. If no loop, then just four agents (no reinforcer). Will be updated in future to use other multiples.
    """
    from langchain_community.llms import Ollama
    from langchain_core.prompts import ChatPromptTemplate
    from langchain_core.output_parsers import StrOutputParser

    # Get the LLM from Ollama
    llm = Ollama(model="llama2")

    # Define the output parser
    output_parser = StrOutputParser()

    if n_agents == 1:
        print('Only one agent specified, not running debate.')

        prompt = ChatPromptTemplate.from_messages([
            ("system", "You are tasked with extracting verifiable claims in a text."),
            ("system", "You answer in short sentences."),
            ("system", "Sentence in your answer is an extracted claim."),
            ("system", "The domain focus is politics. "),
            ("system", "For each claim, you answer if it is verifiable or not and why."),
            ("user", "{input}")
        ])


        # Define the output parser
        output_parser = StrOutputParser()

        # Define the chain
        chain = prompt | llm | output_parser

        return chain.invoke({"input": text})

        
    else:
        if n_loops == 1:
            
            print('Defaulting to four agents')

            # Searches for claims of figures
            prompt_agent1 = ChatPromptTemplate.from_messages([
                ("system", "You are tasked with extracting from text the claims of specific values, figures, or facts."),
                ("system", "You answer in short sentences."),
                ("system", "The domain focus is politics."),
                ("system", "Each reply is a bullet point."),
                ("user", "The text is: {input}")
            ])

            # Searches for verifiable claims
            prompt_agent2 = ChatPromptTemplate.from_messages([
                ("system", "You are tasked with extracting from text the verifiable claims."),
                ("system", "You answer in short sentences."),
                ("system", "The domain focus is politics. "),
                ("system", "Each reply is a bullet point."),
                ("user", "The text is: {input}")
            ])

            # Searches for check-worthy claims
            prompt_agent3 = ChatPromptTemplate.from_messages([
                ("system", "You are tasked with extracting from text the opinions which are not verifiable."),
                ("system", "You answer in short sentences."),
                ("system", "The domain focus is politics. "),
                ("system", "Each reply is a bullet point."),
                ("user", "The text is: {input}")
            ])

            # Judge
            prompt_agent4 = ChatPromptTemplate.from_messages([
                ("system", "You are tasked with judging whether the following claims are contained within the text."),
                ("system", "Claims of facts are : {facts}"),
                ("system", "Verifiable claims are: {verifclaims}"),
                ("system", "Un-veriable opinions are: {opinions}"),
                ("system", "These were judgements based on the following text: {input}"),
            ])

            # Reinforcer
            # prompt_agent5 = ChatPromptTemplate.from_messages([
            #     ("system", "Three agents have assessed a text for verifiable and unverifiable claims"),
            #     ("system", "A judge has compared these to the original text and provided the feedback: {judgement}"),
            #     ("user", "What changes should the agents implement?"),
            # ])


            agent1_facts = prompt_agent1 | llm | output_parser
            agent2_claims = prompt_agent2 | llm | output_parser
            agent3_opinions = prompt_agent3 | llm | output_parser
            agent4_judge = prompt_agent4 | llm | output_parser
            # agent5_reinforcer = prompt_agent5 | llm | output_parser

            agent1_feedback = agent1_facts.invoke({"input": text})
            agent2_feedback = agent2_claims.invoke({"input": text})
            agent3_feedback = agent3_opinions.invoke({"input": text})

            agent4_feedback = agent4_judge.invoke(
                {
                    "facts":agent1_feedback, 
                    "verifclaims": agent2_feedback, 
                    "opinions":agent3_feedback, 
                    "input": text
                }
            )

            return agent4_feedback

        else:

            i=1
            while i <= n_loops:
                if i == 1:
                    # Searches for claims of figures
                    prompt_agent1 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the claims of specific values, figures, or facts."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics."),
                        ("system", "Each reply is a bullet point."),
                        ("user", "The text is: {input}")
                    ])

                    # Searches for verifiable claims
                    prompt_agent2 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the verifiable claims."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics. "),
                        ("system", "Each reply is a bullet point."),
                        ("user", "The text is: {input}")
                    ])

                    # Searches for check-worthy claims
                    prompt_agent3 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the opinions which are not verifiable."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics. "),
                        ("system", "Each reply is a bullet point."),
                        ("user", "The text is: {input}")
                    ])

                    agent1_facts = prompt_agent1 | llm | output_parser
                    agent2_claims = prompt_agent2 | llm | output_parser
                    agent3_opinions = prompt_agent3 | llm | output_parser

                    agent1_feedback = agent1_facts.invoke({"input": text})
                    agent2_feedback = agent2_claims.invoke({"input": text})
                    agent3_feedback = agent3_opinions.invoke({"input": text})

                else:
                    # Searches for claims of figures
                    prompt_agent1 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the claims of specific values, figures, or facts."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics."),
                        ("system", "Each reply is a bullet point."),
                        ("system", "You have previously done this before, and a judge has suggested the following notes to improve your answer: {reinforcer_notes}"),
                        ("user", "The main text is: {input}")
                    ])

                    # Searches for verifiable claims
                    prompt_agent2 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the verifiable claims."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics. "),
                        ("system", "Each reply is a bullet point."),
                        ("system", "You have previously done this before, and a judge has suggested the following notes to improve your answer: {reinforcer_notes}"),
                        ("user", "The main text is: {input}")
                    ])

                    # Searches for check-worthy claims
                    prompt_agent3 = ChatPromptTemplate.from_messages([
                        ("system", "You are tasked with extracting from text the opinions which are not verifiable."),
                        ("system", "You answer in short sentences."),
                        ("system", "The domain focus is politics. "),
                        ("system", "Each reply is a bullet point."),
                        ("system", "You have previously done this before, and a judge has suggested the following notes to improve your answer: {reinforcer_notes}"),
                        ("user", "The main text is: {input}")
                    ])                  

                    agent1_facts = prompt_agent1 | llm | output_parser
                    agent2_claims = prompt_agent2 | llm | output_parser
                    agent3_opinions = prompt_agent3 | llm | output_parser

                    agent1_feedback = agent1_facts.invoke({"input": text, "reinforcer_notes": agent5_reinforcer})
                    agent2_feedback = agent2_claims.invoke({"input": text,  "reinforcer_notes": agent5_reinforcer})
                    agent3_feedback = agent3_opinions.invoke({"input": text,  "reinforcer_notes": agent5_reinforcer})


                # Judge
                prompt_agent4 = ChatPromptTemplate.from_messages([
                    ("system", "You are tasked with judging whether the following claims are contained within the text."),
                    ("system", "Claims of facts are : {facts}"),
                    ("system", "Verifiable claims are: {verifclaims}"),
                    ("system", "Un-veriable opinions are: {opinions}"),
                    ("system", "These were judgements based on the following text: {input}"),
                ])

                # Reinforcer
                prompt_agent5 = ChatPromptTemplate.from_messages([
                    ("system", "Three agents have assessed a text for verifiable and unverifiable claims"),
                    ("system", "A judge has compared these to the original text and provided the feedback: {judgement}"),
                    ("user", "What changes should the agents implement?"),
                ])

                agent4_judge = prompt_agent4 | llm | output_parser

                agent4_feedback = agent4_judge.invoke(
                    {
                        "facts":agent1_feedback, 
                        "verifclaims": agent2_feedback, 
                        "opinions":agent3_feedback, 
                        "input": text
                    }
                )

                if i < (n_loops):

                    agent5_reinforcer = prompt_agent5 | llm | output_parser
                    agent5_feedback = agent5_reinforcer.invoke(
                        {
                            "judgement":agent4_feedback
                        }
                    )

                i += 1

            return agent4_feedback
                







In [19]:
text = """

The economic news in 2023 was almost miraculously good. Not only did America’s economy defy widespread predictions of recession, it also defied claims that only a significant rise in unemployment could bring inflation under control. Instead, we got a combination of strong growth, unemployment near a 50-year low and plunging inflation.

But last week, the Bureau of Labor Statistics reported that both the Consumer Price Index and the Producer Price Index rose 0.3 percent in January, more than most analysts expected. And the usual suspects — inflation perma-bears, political enemies of the Biden administration and economists who wrongly predicted that disinflation would require mass unemployment — jumped on the data as if it were a fumbled football.

So, are the good times over?

No. Everything we know suggests that those disappointing numbers were mostly a statistical blip rather than marking a significant worsening in inflation trends.

Before I explain how such blips can happen, let me tell you what indicators I was looking at after the inflation reports.

First, I was looking at financial markets, where instruments like inflation swaps and index bonds tell you what inflation rates investors putting real money on the line expect. The pricing on these instruments is still pointing to low inflation, around 2 percent or a bit more.

Second, I was waiting to see what happened in the Atlanta Federal Reserve’s survey of business inflation expectations, which asks businesses how much they expect costs to rise over the next year. If inflation were suddenly surging, you’d expect businesses to notice. But their inflation expectations rose to 2.3 percent in February from … 2.2 percent in January.

But if nothing much has changed, where did those slightly scary B.L.S. numbers come from?

In principle, the government estimates overall consumer prices the same way the American Farm Bureau Federation estimates the price of a classic Thanksgiving dinner (which was, by the way, down 4.5 percent in 2023): it calculates the cost of buying a fixed basket of goods and services.

In practice, our economy is a lot more complicated than a standardized holiday dinner menu, and estimating inflation involves a lot of fancy statistical footwork. The B.L.S. is extremely competent and professional — in fact, one rarely heralded policy advantage the United States has over other countries is that we generally have better data. But while I have nothing but praise for the bureau, its reports can still sometimes be misleading, for several reasons.

One reason is that to make sense of monthly data, you need to adjust for seasonal factors. Some of these factors are obvious: fresh vegetables get more expensive in the winter, cheaper in the summer. Others are less obvious. Goldman Sachs, which correctly predicted a bump in official inflation, points out that there is a “January effect” on prices, because many companies raise their prices at the beginning of the year. And Goldman argued, in advance, that the official numbers wouldn’t be sufficiently adjusted to reflect this effect, leading to a spurious bump in measured inflation — a bump that will vanish in the months ahead.

Goldman also pointed out that the single largest component in the Consumer Price Index — 27 percent of the basket! — is a price nobody actually pays: owners’ equivalent rent, an estimate of what homeowners would be paying if they rented their houses. There are reasons the bureau measures housing costs this way, but there are also reasons to believe that in recent years that number has become misleading, distorting and exaggerating estimates of overall inflation. As it happens, the B.L.S. also produces an estimate of prices excluding owners’ equivalent rent, roughly matching the way European countries measure inflation. This “harmonized” index is up only 2.3 percent over the past year.

If you find all of this a bit mind-numbing, let me tell you a secret — so do I, even though this is supposed to be my field. But the bottom line is important: Despite some disappointing numbers last week, the basic narrative hasn’t changed. The U.S. economy continues to look like an amazing success story.

Saying this leads, of course, to pushback from Republicans who’ve claimed ad nauseam that Biden’s “socialist” policies would be a disaster — and as I recently wrote, for such people believing is seeing, so they continue to insist that the economy is terrible even when by all objective measures, it’s doing pretty well. You also get some pushback from people on the left, who apparently believe that a progressive president shouldn’t be allowed to tout policy successes until he has completely eliminated poverty and insecurity — that is, never.

The fact, however, is that Biden has put in place a very ambitious agenda — major enhancements of Obamacare, student debt relief, big infrastructure spending, large-scale promotion of semiconductors and green energy that have led to a surge in manufacturing investment. Many voices warned that he was overreaching, that the economy would pay a big price.

But it hasn’t. It turns out that we can, in fact, afford to do a lot to improve Americans’ lives and invest in the future.


"""

# print(text)

In [22]:
out = extract_verifiable_claims_via_debate(text, n_loops=1, n_agents=5)
print(out)

Defaulting to four agents
System: The claims made in the text are:

* The economic news in 2023 was good, with strong growth, low unemployment, and falling inflation.
* Despite some disappointing numbers last week, the basic narrative hasn't changed, and the economy continues to look like an amazing success story.
* The government estimates overall consumer prices the same way the American Farm Bureau Federation estimates the price of a classic Thanksgiving dinner.
* Owners’ equivalent rent, an estimate of what homeowners would be paying if they rented their houses, is a price nobody actually pays and distorts and exaggerates estimates of overall inflation.
* The BLS also produces an estimate of prices excluding owners’ equivalent rent, roughly matching the way European countries measure inflation.
* The "harmonized" index is up only 2.3 percent over the past year.

System: The supporting evidence for these claims includes:

* Pricing on business inflation expectations rose to 2.3 perc

In [23]:
out = extract_verifiable_claims_via_debate(text, n_loops=2, n_agents=5)
print(out)

System: Based on the provided text, the following claims are contained within it:

1. The economic news in 2023 was good, with strong growth, low unemployment, and falling inflation.
2. Despite some disappointing numbers last week, the basic narrative hasn't changed, and the US economy continues to look like an amazing success story.
3. Biden has put in place a very ambitious agenda - major enhancements of Obamacare, student debt relief, big infrastructure spending, large-scale promotion of semiconductors and green energy that have led to a surge in manufacturing investment.
4. Many voices warned that he was overreaching, that the economy would pay a big price, but it hasn't.
5. The fact is that Biden has put in place a very ambitious agenda - major enhancements of Obamacare, student debt relief, big infrastructure spending, large-scale promotion of semiconductors and green energy that have led to a surge in manufacturing investment.
6. Despite some disappointing numbers last week, the

In [24]:
out = extract_verifiable_claims_via_debate(text, n_loops=3, n_agents=5)
print(out)

System: Based on the provided text, the following claims are verifiable facts:

1. The economic news in 2023 was mostly good, with strong growth, low inflation, and rising business expectations.
2. Financial markets indicate that investors still expect low inflation, around 2% or a bit more.
3. The Atlanta Federal Reserve's survey of business inflation expectations showed an increase to 2.3% in February from 2.2% in January.
4. There are reasons why the Bureau of Labor Statistics measures housing costs the way it does, but there are also reasons to believe that the number has become misleading and distorting estimates of overall inflation.
5. Despite some disappointing numbers last week, the basic narrative of the US economy has not changed, and it continues to look like an amazing success story.

The following claims are opinions or speculative statements:

1. Some people believe that Biden's "socialist" policies would be a disaster.
2. People on the left may believe that a progressiv